In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
# for ensemble method use
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.feature_selection import chi2

# Read the data
Test_Target = ["SUSPECT_ARRESTED_FLAG", "STOP_WAS_INITIATED",
               "OBSERVED_DURATION_MINUTES",
               "STOP_DURATION_MINUTES", "FIREARM_FLAG",
               "FRISKED_FLAG", "SEARCHED_FLAG","SUSPECTED_CRIME_DESCRIPTION"]
Test1 = pd.read_excel("sqf2022.xlsx", usecols=Test_Target)

Test1['FIREARM_FLAG'] = Test1['FIREARM_FLAG'].replace('(null)', 'No')
Test1.replace('(null)', np.nan, inplace=True)
print(Test1.isna().sum())

lb = LabelBinarizer()
Test1["SUSPECT_ARRESTED_FLAG"] = lb.fit_transform(Test1["SUSPECT_ARRESTED_FLAG"])
Test1["FIREARM_FLAG"] = lb.fit_transform(Test1["FIREARM_FLAG"])
Test1["FRISKED_FLAG"] = lb.fit_transform(Test1["FRISKED_FLAG"])
Test1["SEARCHED_FLAG"] = lb.fit_transform(Test1["SEARCHED_FLAG"])


Test1["SUSPECT_ARRESTED_FLAG"].fillna(Test1["SUSPECT_ARRESTED_FLAG"].median,inplace=True)
Test1["FIREARM_FLAG"].fillna(Test1["FIREARM_FLAG"].median,inplace=True)
Test1["FRISKED_FLAG"].fillna(Test1["FRISKED_FLAG"].median,inplace=True)
Test1["SEARCHED_FLAG"].fillna(Test1["SEARCHED_FLAG"].median,inplace=True)



OneHot_Encode_feature = ["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION"]
Label_En = OneHotEncoder(handle_unknown="ignore")
Feature_Encoded = Label_En.fit_transform(Test1[OneHot_Encode_feature])
Label_df = pd.DataFrame(Feature_Encoded.toarray(), columns=Label_En.get_feature_names_out(OneHot_Encode_feature))
#Label_df.fillna(value=0, inplace=True)
#Label_df.fillna(median)




Test1 = pd.concat([Test1, Label_df], axis=1)
# drop the original categorical feature column
Test1 = Test1.drop(columns=OneHot_Encode_feature)

train_data, test_data = train_test_split(Test1, test_size=0.2, random_state=42)

# Compute the correlation matrix
corr_matrix = train_data.corr()



# Select the top k features with highest correlation using SelectKBest
k = 5  # Number of features to select
selector = SelectKBest(score_func=f_classif, k=k)
selector.fit(train_data.drop("SUSPECT_ARRESTED_FLAG", axis=1), train_data["SUSPECT_ARRESTED_FLAG"])

# Get the selected feature indices
selected_feature_indices = selector.get_support(indices=True)

# Get the selected feature names
selected_feature_names = train_data.drop("SUSPECT_ARRESTED_FLAG", axis=1).columns[selected_feature_indices]

# Print the selected feature names
print()
print("Selected Features:?>>>>>>",selected_feature_names)

STOP_WAS_INITIATED             0
OBSERVED_DURATION_MINUTES      0
SUSPECTED_CRIME_DESCRIPTION    0
STOP_DURATION_MINUTES          0
SUSPECT_ARRESTED_FLAG          0
FRISKED_FLAG                   0
SEARCHED_FLAG                  0
FIREARM_FLAG                   0
dtype: int64

Selected Features:?>>>>>> Index(['SEARCHED_FLAG', 'FIREARM_FLAG',
       'STOP_WAS_INITIATED_Based on Self Initiated',
       'SUSPECTED_CRIME_DESCRIPTION_CPW',
       'SUSPECTED_CRIME_DESCRIPTION_PETIT LARCENY'],
      dtype='object')
